In [0]:
df= spark.read.format('csv')\
    .option("header",True)\
    .option("inferSchema",True)\
    .load("/Volumes/pyspark_proj/source/source_data/customers/")
display(df)

In [0]:
schema_customer = df.schema

### SPARK STREAMING

In [0]:
entities = ['customers','trips','locations','payments','drivers','vehicles']



In [0]:
#Dynamic solution for data streaming 
for entity in entities:
    
    df_batch= spark.read.format('csv')\
        .option("header",True)\
        .option("inferSchema",True)\
        .load(f"/Volumes/pyspark_proj/source/source_data/{entity}/")
    
    entity_schema = df_batch.schema

    df = spark.readStream.format('csv')\
        .option("header",True)\
        .schema(entity_schema)\
        .load(f"/Volumes/pyspark_proj/source/source_data/{entity}/")


    df.writeStream.format("delta").outputMode("append")\
    .option("checkpointLocation",f"/Volumes/pyspark_proj/bronze/checkpoint/{entity}")\
    .trigger(once=True)\
    .toTable(f"pyspark_proj.bronze.{entity}")